In [1]:
# Import required libraries
import pandas as pd
import numpy as np

In [2]:
#read the data into pandas dataframes.
admissionsDf = pd.read_csv("Data/ADMISSIONS.csv")
diagnosesDf = pd.read_csv("Data/DIAGNOSES_ICD.csv")
eventsDf = pd.read_csv("Data/NOTEEVENTS.csv", dtype={"CHARTTIME":"string", "STORETIME":"string"})
patientsDf = pd.read_csv("Data/PATIENTS.csv")

In [3]:
#sort admissions dataframe by subject id and date and drop duplicate subject id so that only patient record for first visit is retained
admissionsDf.sort_values(["SUBJECT_ID", "ADMITTIME"], ascending=[True, True], inplace=True)
admissionsDf.drop_duplicates(subset=["SUBJECT_ID"], inplace=True)

dobDf = patientsDf[["SUBJECT_ID", "DOB"]] #create a dataframe of patient id and corresponding date of birth
admissionsDf1 = pd.merge(admissionsDf, dobDf, how="left", on="SUBJECT_ID") #merge the date of births to the admissions dataframe and reassign the admissions dataframe
admissionsDf1["ADMITTIME"] = pd.to_datetime(admissionsDf1["ADMITTIME"]) #convert admit time to datetime
admissionsDf1["DOB"] = pd.to_datetime(admissionsDf1["DOB"]) #convert DOB to datetime 

admissionsDf1['AGE'] = (admissionsDf1["ADMITTIME"].values - admissionsDf1["DOB"].values) / np.timedelta64(1,"D") // 365 #calculate the age of each patient at the time of admission


In [30]:
#find the patient id for all patients < 15 years old
under15 = admissionsDf1.loc[abs(admissionsDf1["AGE"]) < 15.0]
under15Patients = list(under15["SUBJECT_ID"]) #7,875 patients under 15 years

In [31]:
#drop under15 patients from admissions, diagnoses, events and patients
admissionsFiltered = admissionsDf1[~admissionsDf1.SUBJECT_ID.isin(under15Patients)]
diagnosesFiltered = diagnosesDf[~diagnosesDf.SUBJECT_ID.isin(under15Patients)]
eventsFiltered = eventsDf[~eventsDf.SUBJECT_ID.isin(under15Patients)]
patientsFiltered = patientsDf[~patientsDf.SUBJECT_ID.isin(under15Patients)] #38,645 adult (>15 years) patients

#Note: the FarSight paper incorrectly states there are 7,704 distinct patients (page 1155)

In [74]:
#drop events with known errors. i.e. ISERROR = 1
eventsNoError = eventsFiltered.loc[eventsFiltered.ISERROR != 1]
#drop duplicate events from the filtered dataframe
eventsNoDuplicate = eventsNoError.drop_duplicates() 

#create list of patients from events list which have no errors
patientsNoErrors = list(events.SUBJECT_ID.unique())

In [76]:
#select patient id and hospital admission code from admissions dataframe and use to merge left with diagnoses and events
patientHadmCode = admissionsFiltered[["SUBJECT_ID", "HADM_ID"]]
diagnosesFiltered1 = pd.merge(patientHadmCode, diagnosesFiltered, how="left", on=["SUBJECT_ID", "HADM_ID"])
eventsFiltered1 = pd.merge(patientHadmCode, eventsNoDuplicate, how="left", on=["SUBJECT_ID", "HADM_ID"])

In [77]:
#prepare the final dataframes for further analysis
admissions = admissionsFiltered[admissionsFiltered.SUBJECT_ID.isin(patientsNoErrors)] #final admissions dataframe to be used for further analysis
diagnoses = diagnosesFiltered1[diagnosesFiltered1.SUBJECT_ID.isin(patientsNoErrors)] #final diagnoses dataframe to be used for further analysis
patients = patientsFiltered[patientsFiltered.SUBJECT_ID.isin(patientsNoErrors)] #final patients dataframe to be used for further analysis
events = eventsFiltered1[eventsFiltered1.SUBJECT_ID.isin(patientsNoErrors)] #final events dataframe to be used for further analysis


In [ ]:
###cells below are not included in the project code

In [78]:
patients.shape

(38645, 8)

In [87]:
events.shape

(1079308, 11)

In [88]:
diagnoses.shape

(429770, 5)

In [79]:
admissions.SUBJECT_ID.nunique()

38645

In [80]:
diagnoses.SUBJECT_ID.nunique()

38645

In [81]:
events.SUBJECT_ID.nunique()

38645

In [82]:
display(patients.head(10))

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0
5,239,255,M,2109-08-05 00:00:00,NaN,NaN,NaN,0
6,240,256,M,2086-07-31 00:00:00,NaN,NaN,NaN,0
7,241,257,F,2031-04-03 00:00:00,2121-07-08 00:00:00,2121-07-08 00:00:00,2121-07-08 00:00:00,1
10,244,261,M,2025-08-04 00:00:00,2102-06-29 00:00:00,2102-06-29 00:00:00,2102-06-29 00:00:00,1
11,245,262,M,2090-01-05 00:00:00,NaN,NaN,NaN,0


In [83]:
display(admissions.head(10))

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,...,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,DOB,AGE
1,2,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,...,CATHOLIC,MARRIED,WHITE,2101-10-20 17:09:00,2101-10-20 19:24:00,HYPOTENSION,0,1,2025-04-11,76.0
2,3,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,Private,...,PROTESTANT QUAKER,SINGLE,WHITE,2191-03-15 13:10:00,2191-03-16 01:10:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0,1,2143-05-12,47.0
4,5,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,...,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,CHRONIC RENAL FAILURE/SDA,0,1,2109-06-21,65.0
7,8,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicaid,...,UNOBTAINABLE,NaN,UNKNOWN/NOT SPECIFIED,2149-11-09 11:13:00,2149-11-09 13:18:00,HEMORRHAGIC CVA,1,1,2108-01-26,41.0
9,10,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Private,...,OTHER,MARRIED,WHITE,2178-04-15 20:46:00,2178-04-16 06:53:00,BRAIN MASS,0,1,2128-02-22,50.0
10,11,12,112213,2104-08-07 10:15:00,2104-08-20 02:57:00,2104-08-20 02:57:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,DEAD/EXPIRED,Medicare,...,JEWISH,MARRIED,WHITE,NaN,NaN,PANCREATIC CANCER/SDA,1,1,2032-03-24,72.0
11,12,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicaid,...,OTHER,NaN,WHITE,NaN,NaN,CORONARY ARTERY DISEASE,0,1,2127-02-27,39.0
13,14,17,194023,2134-12-27 07:15:00,2134-12-31 16:05:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,...,CATHOLIC,MARRIED,WHITE,NaN,NaN,PATIENT FORAMEN OVALE\ PATENT FORAMEN OVALE MI...,0,1,2087-07-14,47.0
14,16,18,188822,2167-10-02 11:18:00,2167-10-04 16:15:00,NaN,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME,Private,...,CATHOLIC,MARRIED,WHITE,NaN,NaN,HYPOGLYCEMIA;SEIZURES,0,1,2116-11-29,50.0
15,17,19,109235,2108-08-05 16:25:00,2108-08-11 11:29:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,...,UNOBTAINABLE,NaN,WHITE,2108-08-05 08:01:00,2108-08-05 20:12:00,C-2 FRACTURE,0,1,1808-08-05,-285.0


In [84]:
display(diagnoses.head(10))

,SUBJECT_ID,HADM_ID,ROW_ID,SEQ_NUM,ICD9_CODE
0,3,145834,4,1.0,0389
1,3,145834,5,2.0,78559
2,3,145834,6,3.0,5849
3,3,145834,7,4.0,4275
4,3,145834,8,5.0,41071
5,3,145834,9,6.0,4280
6,3,145834,10,7.0,6826
7,3,145834,11,8.0,4254
8,3,145834,12,9.0,2639
9,4,185777,13,1.0,042


In [85]:
display(events.head(10))

,SUBJECT_ID,HADM_ID,ROW_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,3,145834,44005.0,2101-10-31,<NA>,<NA>,Discharge summary,Report,NaN,NaN,Admission Date: [**2101-10-20**] Discharg...
1,3,145834,94503.0,2101-10-21,<NA>,<NA>,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: S/P Car...
2,3,145834,94504.0,2101-10-21,<NA>,<NA>,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Pericar...
3,3,145834,94502.0,2101-10-21,<NA>,<NA>,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Left ve...
4,3,145834,272785.0,2101-10-27,<NA>,<NA>,ECG,Report,NaN,NaN,Sinus rhythm\nP-R interval increased\nLate R w...
5,3,145834,272786.0,2101-10-25,<NA>,<NA>,ECG,Report,NaN,NaN,Technically difficult study\nSinus rhythm with...
6,3,145834,272787.0,2101-10-26,<NA>,<NA>,ECG,Report,NaN,NaN,Sinus rhythm\nP-R interval increased\nLate R w...
7,3,145834,272788.0,2101-10-23,<NA>,<NA>,ECG,Report,NaN,NaN,Sinus rhythm\nLeft atrial abnormality\nNondiag...
8,3,145834,272789.0,2101-10-22,<NA>,<NA>,ECG,Report,NaN,NaN,Sinus rhythm with premature ventricular contra...
9,3,145834,272790.0,2101-10-20,<NA>,<NA>,ECG,Report,NaN,NaN,Sinus rhythm\nInferior/lateral T changes are n...


In [86]:
display(events.loc[events.SUBJECT_ID==249]) #double check this and filter if required so that only events for the first admission date is retained

,SUBJECT_ID,HADM_ID,ROW_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
4214,249,116935,48758.0,2149-12-31,<NA>,<NA>,Discharge summary,Report,NaN,NaN,Admission Date: [**2149-12-17**] ...
4215,249,116935,100078.0,2149-12-19,<NA>,<NA>,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Chronic...
4216,249,116935,274072.0,2149-12-24,<NA>,<NA>,ECG,Report,NaN,NaN,Atrial fibrillation\nAtrial premature complex ...
4217,249,116935,274314.0,2149-12-17,<NA>,<NA>,ECG,Report,NaN,NaN,Atrial fibrillation with rapid ventricular res...
4218,249,116935,274308.0,2149-12-19,<NA>,<NA>,ECG,Report,NaN,NaN,Atrial fibrillation\nLeft axis deviation\nAnte...
4219,249,116935,274309.0,2149-12-18,<NA>,<NA>,ECG,Report,NaN,NaN,Atrial fibrillation\nLeft axis deviation - pos...
4220,249,116935,274310.0,2149-12-18,<NA>,<NA>,ECG,Report,NaN,NaN,Atrial fibrillation\nLeft axis deviation - lef...
4221,249,116935,274311.0,2149-12-18,<NA>,<NA>,ECG,Report,NaN,NaN,Atrial fibrillation\nMarked left axis deviatio...
4222,249,116935,274312.0,2149-12-17,<NA>,<NA>,ECG,Report,NaN,NaN,Atrial fibrillation\nLeft axis deviation - lef...
4223,249,116935,274313.0,2149-12-17,<NA>,<NA>,ECG,Report,NaN,NaN,Atrial fibrillation with rapid ventricular res...
